# BCGNet Demo

In [10]:
%pip install mne


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
!git clone https://github.com/htylab/BCGNet.git
%cd BCGNet

Cloning into 'BCGNet'...
remote: Enumerating objects: 355, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (71/71), done.
^Cceiving objects:  20% (71/355), 52.50 MiB | 574.00 KiB/s  
fetch-pack: unexpected disconnect while reading sideband packet
[Errno 2] No such file or directory: 'BCGNet'
/Users/takala/Documents/GitHub/BCGNet/BCGNet/BCGNet


In [ ]:
!pip install numpy~=1.19.5

## Path setup

### Option 2: In Python

For the purpose of this demo, we will set all the variables listed above in the Jupyter notebook. Additionally, here we will set them as pathlib objects instead of strings for convenience.

In [ ]:
import os
import tqdm

from pathlib import Path
from config import get_config
from session import Session

import glob
from scipy.io import savemat
import os
from os.path import *
import time
result_dir = r'./BCGNET_result'
os.makedirs(result_dir, exist_ok=True)

ffs = glob.glob(r"/Users/takala/Documents/GitHub/BCGunet/data/noscan/*/analysis/*.mat")

for f in tqdm.tqdm(ffs):
    time_list = []

    print(f)
    d_root = Path(os.getcwd())
    cfg = get_config(filename=d_root / 'config' / 'default_config.yaml')
    cfg.d_root = d_root
    cfg.d_model = ''
    cfg.d_output = ''
    cfg.d_eval = None
    cfg.str_eval = None
    cfg.d_data = Path(f)

    t = time.time()
    s1 = Session(str_sub='sub34', vec_idx_run=[1], str_arch='default_rnn_model',
                 random_seed=1997, verbose=1, overwrite=False, cv_mode=False, num_fold=5, cfg=cfg)
    # loads all dataset
    s1.load_all_dataset()
    # preform preprocessing of all dataset and initialize model
    s1.prepare_training()
    s1.train()
    s1.clean()
    result = s1.vec_dataset[0].orig_cleaned_dataset.get_data()
    runtime = time.time() - t
    mat_f1 = join(result_dir, basename(dirname(dirname(f))) + '-' + basename(f))
    mat_f2 = f.replace('.mat', '_bcgnet.mat')

    mat = dict()
    mat['bcgnet'] = result[:31, :]
    savemat(mat_f1, mat, do_compression=True)
    savemat(mat_f2, mat, do_compression=True)
    time_list.append(runtime)
    print(runtime)